In [43]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from fuzzywuzzy import fuzz
import Levenshtein
import random

In [41]:
import keras
from keras.layers import Dense, Dropout, LSTM, Conv2D, Conv1D, MaxPooling1D, MaxPooling2D, Flatten, ZeroPadding1D
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

/Users/zswitten/.pyenv/versions/2.7.12/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('Hemolysis/Cleaned_hemolytic_data.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.columns = [c.lower() for c in df.columns]
df.sequence = df.sequence.str.upper()

In [16]:
def knn_toxicities(peptide_row, df, k, min_uncertainty):
    sequence = peptide_row.sequence
    distances = df[df.uncertainty < min_uncertainty][df.sequence != sequence].sequence.apply(lambda x: Levenshtein.distance(x, sequence))
    bestk = df.loc[distances.sort_values().iloc[:k]]
    avg_toxicity = np.mean(bestk.log10_hc50)
    return avg_toxicity


In [17]:
for i, row in df.iterrows():
    bestk = knn_toxicities(row, df, 5, 0.1)
    df.at[i, 'bestk'] = bestk

/Users/zswitten/.pyenv/versions/2.7.12/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
df[['log10_hc50', 'bestk']].corr()

,log10_hc50,bestk
log10_hc50,1.00000,0.03919
bestk,0.03919,1.00000


In [21]:
df[df.uncertainty < 0.1][['log10_hc50', 'bestk']].corr()

,log10_hc50,bestk
log10_hc50,1.000000,0.035214
bestk,0.035214,1.000000


In [22]:
df[df.uncertainty > 0.1][['log10_hc50', 'bestk']].corr()

,log10_hc50,bestk
log10_hc50,1.000000,0.062517
bestk,0.062517,1.000000


# Using CNN Architecture from MIC Prediction

In [30]:
CHARACTER_DICT = set([character for sequence in df.sequence for character in sequence])
MAX_SEQUENCE_LENGTH = int(df.sequence.str.len().describe(percentiles=[0.95])['95%'])

# Each amino acid its own group
character_to_index = {
    (character): i
    for i, character in enumerate(CHARACTER_DICT)
}

# Group them together heavily
"""character_to_index = {
    ('R', 'K', 'H'): 0,
    ('D', 'E'): 1,
    ('S', 'T', 'N', 'Q', 'C'): 2,
    ('A', 'V', 'I', 'L', 'M', 'F', 'Y', 'W', 'P', 'G'): 3,
}

# Group them together more sparingly
character_to_index = {
    ('R'): 0,
    ('H'): 1,
    ('K'): 2,
    ('D', 'E'): 3,
    ('S', 'T', 'N', 'Q', 'C'): 4,
    ('G', 'P'): 5,
    ('A', 'V', 'I', 'L', 'M'): 6,
    ('F', 'Y', 'W'): 7,
}"""

index2character = {
    value: key
    for key, value in character_to_index.items()
}

def sequence_to_vector(sequence):
    default = np.zeros([MAX_SEQUENCE_LENGTH, len(character_to_index)])
    for i, character in enumerate(sequence[:MAX_SEQUENCE_LENGTH]):
        default[i][character_to_index[character]] = 1
    return default

def row_to_vector(row, shuffle_sequence=False):
    sequence = list(row['sequence'])
    if shuffle_sequence:
        random.shuffle(sequence)
    return sequence_to_vector(sequence)

In [35]:
vectors = []
SHUFFLE_SEQUENCE = False
for row in df.iterrows():
    vectors.append(row_to_vector(row[1], shuffle_sequence=SHUFFLE_SEQUENCE))

vectors = np.array(vectors)

labels = np.array(df.log10_hc50)

In [39]:
def generate_train_test_splits(
        vectors, labels,
        extra_training_vectors=[], extra_training_labels=[], extra_sample_weights=[],
        cutoff=0.85
):
    cutoff = int(cutoff * len(labels))
    idx = range(len(vectors))
    random.shuffle(idx)
    reordered_vectors = vectors[idx]
    reordered_labels = labels[idx]
    reordered_sample_weights = sample_weights[idx]
    if len(extra_training_vectors) > 0:
        train_x = np.concatenate((reordered_vectors[:cutoff], extra_training_vectors))
        train_y = np.concatenate((reordered_labels[:cutoff], extra_training_labels))
        train_sample_weights = np.concatenate((reordered_sample_weights[:cutoff], pa_sample_weights))
    else:
        train_x = reordered_vectors[:cutoff]
        train_y = reordered_labels[:cutoff]
        train_sample_weights = reordered_sample_weights[:cutoff]
    test_x = reordered_vectors[cutoff:]
    test_y = reordered_labels[cutoff:]
    return train_x, train_y, test_x, test_y, train_sample_weights

In [45]:
# Convolutional NN
def conv_model():
    model = keras.models.Sequential()
    model.add(ZeroPadding1D(
        5, input_shape = (MAX_SEQUENCE_LENGTH, len(character_to_index))
    ))
    model.add(Conv1D(
        64,
        kernel_size = 5,
        strides = 1,
        activation = 'relu',
        #input_shape = (MAX_SEQUENCE_LENGTH, len(character_to_index) + 1)
    ))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    #model.add(Dropout(0.5))
    model.add(Conv1D(64, 5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    #model.add(Dense(100, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
convmodel = conv_model()
train_x, train_y, test_x, test_y, _ = generate_train_test_splits(vectors, labels)
convmodel.fit(train_x, train_y, batch_size=40, epochs=100)

Epoch 1/100
1170/1170 [==============================] - 1s 511us/step - loss: 1.5147
Epoch 2/100
1170/1170 [==============================] - 0s 160us/step - loss: 0.6160
Epoch 3/100
1170/1170 [==============================] - 0s 158us/step - loss: 0.5672
Epoch 4/100
1170/1170 [==============================] - 0s 157us/step - loss: 0.4749
Epoch 5/100
1170/1170 [==============================] - 0s 157us/step - loss: 0.4389
Epoch 6/100
1170/1170 [==============================] - 0s 155us/step - loss: 0.4280
Epoch 7/100
1170/1170 [==============================] - 0s 159us/step - loss: 0.3891
Epoch 8/100
1170/1170 [==============================] - 0s 154us/step - loss: 0.3734
Epoch 9/100
1170/1170 [==============================] - 0s 174us/step - loss: 0.3318
Epoch 10/100
1170/1170 [==============================] - 0s 162us/step - loss: 0.3267
Epoch 11/100
1170/1170 [==============================] - 0s 157us/step - loss: 0.2892
Epoch 12/100
1170/1170 [============================

In [86]:
print("CNN test error, MSE of log10_hc50")
print(convmodel.evaluate(test_x, test_y))

CNN test error, MSE of log10_hc50
207/207 [==============================] - 0s 79us/step
0.4875335242725225


In [78]:
cnn_preds = convmodel.predict(test_x)
cnn_preds = cnn_preds.reshape((cnn_preds.shape[0]))
pd.DataFrame({'cnn_preds': cnn_preds, 'test_y': test_y}).corr()

,cnn_preds,test_y
cnn_preds,1.000000,0.548227
test_y,0.548227,1.000000
